In [1]:
%matplotlib inline
import sys
import xarray as xr
import numpy as np
import os
import yaml
import math
from datetime import datetime, timedelta
from parcels import FieldSet, Field, VectorField, ParticleSet, JITParticle, ErrorCode, ParcelsRandom, Variable

sys.path.append('/home/jvalenti/MOAD/analysis-jose/Source')

from OP_functions import *
from OP_Kernels import *

local = 0
restart=0

In [2]:
def pandas_deploy(N,MFc,r,dd,dtp):
    #r is radius of particle cloud [m]
    MFc = float(MFc)
    Outfall_deploy = pd.read_csv(N, index_col = [0])
    Pol = list(Outfall_deploy.Population)
    Lat = Outfall_deploy.Lat
    Lon = Outfall_deploy.Lon
    Depth = Outfall_deploy.Depth
    clat = []
    clon = []
    cz = []
    for i,loc in enumerate(Pol):
        for j in range(int(round((loc*250*dtp)/MFc,0))):
            clat.append(Lat.iat[i])
            clon.append(Lon.iat[i])
            try:
                cz.append(float(Depth.iat[i]))
            except ValueError:
                cz.append(2)
    N = len(clat)
    deg2m = 111000 * np.cos(49 * np.pi / 180)
    var = (r / (deg2m * 3))**2
    x_offset, y_offset = np.random.multivariate_normal([0, 0], [[var, 0], [0, var]], [n,N]).T
    z_offset=np.random.random_sample([N]).T*(dd)
    lon = np.zeros([N])
    lat = np.zeros([N])
    z = np.zeros([N])
    for i in range(N):
        lon[i]=(clon[i] + x_offset[i])
        lat[i]=(clat[i] + y_offset[i])
        z[i]=(cz[i] + z_offset[i])
    return lon,lat,z

In [3]:
def get_timestamps(start,length):
    timestamps=[]
    duration = timedelta(days=length)
    for day in range(duration.days):
        timestamps.append([start + timedelta(days=day)])
    return np.array(timestamps, dtype='datetime64')

def find_temp(rootdir):
    dirs=[]
    for file in os.listdir(rootdir):
        d = os.path.join(rootdir, file)
        if os.path.isdir(d):
            dirs.append(d)
    temp=sorted(dirs, key=lambda x: os.path.getctime(x), reverse=True)[:1][0]
    return temp[-12:]

def newest(path):
    files = os.listdir(path)
    paths = [os.path.join(path, basename) for basename in files]
    return max(paths, key=os.path.getctime)


In [4]:
# Define paths
paths = path(local)
#Load configuration from .yaml
param = load_config(['/home/jvalenti/MOAD/analysis-jose/OParcels/yaml/test_outfalls.yaml'])
#Definitions
start = datetime(param['startdate']['year'], param['startdate']['month'], param['startdate']['day']) #Start date
length = param['param']['length'] # Set Time length [days] 
dt = param['param']['dt'] #toggle between - or + to pick backwards or forwards 
N0 = param['param']['N'] # number of deploying locations
n = param['param']['n'] # 1000   # number of particles per location
dmin = param['param']['dmin'] #minimum depth
dd = param['param']['dd'] #max depth difference from dmin
name = param['file']['name'] #name output file
dtp = param['param']['dtp'] #how often particle released in hours
odt = param['param']['odt'] #how often data is recorded
rrr = param['param']['r'] #radious of particle deployment
distr = param['param']['distr']
MFc = param['param']['MFc']
r = param['param']['r']
duration = timedelta(days=length)

In [5]:
lon, lat, z = pandas_deploy(N0,MFc,r,dd,dtp)

In [6]:
#Set start date time and the name of the output file

daterange = [start+timedelta(days=i) for i in range(length)]
fn =  name + '_'.join(d.strftime('%Y%m%d')+'_1n' for d in [start, start+duration]) + '.zarr'
outfile = os.path.join(paths['out'], fn)
####BUILD FIELDS FOR SIMULATION######
#Fill in the list of variables that you want to use as fields
varlist=['U','V','W']
filenames,variables=filename_set(start,length,varlist,local)
dimensions = {'lon': 'glamf', 'lat': 'gphif', 'depth': 'depthw','time': 'time_counter'}
field_set=FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)

#Find file names and variable names
varlist=['US','VS','WL','Diat','Flag','R','T','S','FS','ssh','Bathy','Kz','totdepth','Vol']
filenames,variables=filename_set(start,length,varlist,local)

#Add Stokes Drift fields
dimensions = {'lon': 'longitude', 'lat': 'latitude', 'time': 'time'}
us = Field.from_netcdf(filenames['US'], variables['US'], dimensions,allow_time_extrapolation=True)
vs = Field.from_netcdf(filenames['VS'], variables['VS'], dimensions,allow_time_extrapolation=True)
wl = Field.from_netcdf(filenames['WL'], variables['WL'], dimensions,allow_time_extrapolation=True)
field_set.add_field(us)
field_set.add_field(vs)
field_set.add_field(wl)
field_set.add_vector_field(VectorField("stokes", us, vs, wl))

#Add Vertical diffusivity coefficient field
dimensions = {'lon': 'glamt', 'lat': 'gphit', 'depth': 'depthw','time': 'time_counter'}
Kz = Field.from_netcdf(filenames['Kz'], variables['Kz'], dimensions,allow_time_extrapolation=True)
field_set.add_field(Kz)

#Add fields located at node T
dimensions = {'lon': 'glamt', 'lat': 'gphit', 'depth': 'deptht','time': 'time_counter'}
Diat = Field.from_netcdf(filenames['Diat'], variables['Diat'], dimensions,allow_time_extrapolation=True)
Flag = Field.from_netcdf(filenames['Flag'], variables['Flag'], dimensions,allow_time_extrapolation=True)
R = Field.from_netcdf(filenames['R'], variables['R'], dimensions,allow_time_extrapolation=True)
S = Field.from_netcdf(filenames['S'], variables['S'], dimensions,allow_time_extrapolation=True)
T = Field.from_netcdf(filenames['T'], variables['T'], dimensions,allow_time_extrapolation=True)
field_set.add_field(Diat)
field_set.add_field(Flag)
field_set.add_field(R)
field_set.add_field(S)
field_set.add_field(T)

#Add Bathymetry 2D field
dimensions = {'lon': 'glamt', 'lat': 'gphit'}
Bth = Field.from_netcdf(filenames['Bathy'], variables['Bathy'], dimensions,allow_time_extrapolation=True)
TD = Field.from_netcdf(filenames['totdepth'], variables['totdepth'], dimensions,allow_time_extrapolation=True)
field_set.add_field(Bth)
field_set.add_field(TD)

#Add Volume 3D field
dimensions = {'lon': 'glamt', 'lat': 'gphit', 'depth': 'depthw'}
Vol = Field.from_netcdf(filenames['Vol'], variables['Vol'], dimensions,allow_time_extrapolation=True)
field_set.add_field(Vol)


#Add SSH and Rivers 2D fields
dimensions = {'lon': 'glamt', 'lat': 'gphit','time': 'time_counter'}
SSH = Field.from_netcdf(filenames['ssh'], variables['ssh'], dimensions,allow_time_extrapolation=True)
field_set.add_field(SSH)
Fraser = Field.from_netcdf(filenames['FS'], variables['FS'], dimensions,allow_time_extrapolation=True,timestamps=get_timestamps(start,length))
field_set.add_field(Fraser)

         It will be opened with no decoding. Filling values might be wrongly parsed.


In [7]:
####BUILD Particle typeN######
MPParticle = particle_maker(param)

if restart==1:
    name_temp=find_temp(paths['out'])
    os.system(f"cd {paths['out']} && parcels_convert_npydir_to_netcdf {name_temp}")
    outfile=newest(paths['out'])
    pset = ParticleSet.from_particlefile(field_set, MPParticle,outfile)
else:
    if dtp == 0:
        pset = ParticleSet.from_list(field_set, MPParticle, lon=lon, lat=lat, depth=z,time=start+timedelta(hours=odt))
    else:
        pset = ParticleSet.from_list(field_set, MPParticle, lon=lon, lat=lat, depth=z, repeatdt = timedelta(hours=dtp))

In [8]:
KERNELS =  Advection + pset.Kernel(Buoyancy) + pset.Kernel(Stokes_drift) + pset.Kernel(turb_mix) + pset.Kernel(Displacement) + pset.Kernel(Unbeaching)
pset.execute(KERNELS,
            runtime=duration, 
            dt=dt,
            output_file=pset.ParticleFile(name=outfile, outputdt=timedelta(hours=odt)),
            recovery={ErrorCode.ErrorOutOfBounds: DeleteParticle})

INFO: Compiled ArrayMPParticleAdvectionBuoyancyStokes_driftturb_mixDisplacementUnbeaching ==> /tmp/parcels-2894/libc815e620efd05b8b103bede4c9f96ab2_0.so
INFO: Output files are stored in /home/jvalenti/MOAD/results/Outfalls_202220220101_1n_20220102_1n.zarr.
 40%|███▉      | 34200.0/86400.0 [01:33<02:18, 377.73it/s]<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
 81%|████████▏ | 70200.0/86400.0 [03:51<00:52, 308.62it/s]<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
100%|██████████| 86400.0/86400.0 [05:03<00:00, 284.45it/s]
